In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [57]:
# load econ data
approval = pd.read_csv('../DATA/approval_rating.csv', index_col=False)
gdp = pd.read_csv('../DATA/real_GDP_per_capita.csv', index_col=False)
gdp_change = pd.read_csv('../DATA/real_GDP_per_capita_daily_change.csv', index_col=False)
income = pd.read_csv('../DATA/median_household_income.csv', index_col=False)
income_change = pd.read_csv('../DATA/median_household_income_daily_change.csv', index_col=False)
sp500 = pd.read_csv('../DATA/sp500_historical_data.csv', index_col=False)
sp500_change = pd.read_csv('../DATA/sp500_daily_change.csv', index_col=False)
unemployment = pd.read_csv('../DATA/unemployment_rate.csv', index_col=False)

In [72]:
# rename columns for clean names
gdp.rename(columns={'A939RX0Q048SBEA': 'GDP', 'observation_date': 'observation_date'}, inplace=True)
gdp_change.rename(columns={'Change': 'GDP_Change', 'observation_date': 'observation_date'}, inplace=True)

income.rename(columns={'MEHOINUSA672N': 'Income', 'observation_date': 'observation_date'}, inplace=True)
income_change.rename(columns={'MEHOINUSA672N': 'Income', 'Change': 'Income_Change', 'observation_date': 'observation_date'}, inplace=True)

unemployment.rename(columns={'UNRATE': 'Unemployment', 'observation_date': 'observation_date'}, inplace=True)
sp500.rename(columns={'Close': 'SP500_Close'}, inplace=True)
sp500_change.rename(columns={'Close': 'SP500_Close', 'Close_Change': 'SP500_Close_Change'}, inplace=True)


In [73]:
# make dates datetime format
gdp['observation_date'] = pd.to_datetime(gdp['observation_date'])
gdp_change['observation_date'] = pd.to_datetime(gdp_change['observation_date'])
income['observation_date'] = pd.to_datetime(income['observation_date'])
income_change['observation_date'] = pd.to_datetime(income_change['observation_date'])
sp500['Date'] = pd.to_datetime(sp500['Date'], utc=True).dt.tz_localize(None)
sp500_change['Date'] = pd.to_datetime(sp500_change['Date'], utc=True).dt.tz_localize(None)
unemployment['observation_date'] = pd.to_datetime(unemployment['observation_date'])

In [74]:
# sort values for merge_asof()
approval = approval.sort_values(by='Start Date')
gdp = gdp.sort_values(by='observation_date')
gdp_change = gdp_change.sort_values(by='observation_date')
income = income.sort_values(by='observation_date')
income_change = income_change.sort_values(by='observation_date')
unemployment = unemployment.sort_values(by='observation_date')
sp500 = sp500.sort_values(by='Date')
sp500_change = sp500_change.sort_values(by='Date')

In [75]:
approval['Start Date'] = pd.to_datetime(approval['Start Date'])
gdp['observation_date'] = pd.to_datetime(gdp['observation_date'])

df = pd.merge_asof(approval, gdp, left_on='Start Date', right_on='observation_date', direction='backward')
df.drop(columns=['observation_date'], inplace=True)

df = pd.merge_asof(df, gdp_change, left_on='Start Date', right_on='observation_date', direction='backward')
df.drop(columns=['observation_date'], inplace=True)

df = pd.merge_asof(df, income, left_on='Start Date', right_on='observation_date', direction='backward')
df.drop(columns=['observation_date'], inplace=True)

# Drop  Date After Merge
df = pd.merge_asof(df, income_change, left_on='Start Date', right_on='observation_date', direction='backward')
df.drop(columns=['observation_date'], inplace=True)  # ✅ THIS IS THE FIX

df = pd.merge_asof(df, unemployment, left_on='Start Date', right_on='observation_date', direction='backward')
df.drop(columns=['observation_date'], inplace=True)

df = pd.merge_asof(df, sp500, left_on='Start Date', right_on='Date', direction='backward')
df.drop(columns=['Date'], inplace=True)

df = pd.merge_asof(df, sp500_change, left_on='Start Date', right_on='Date', direction='backward')
df.drop(columns=['Date'], inplace=True)

print(df)

     Start Date    End Date  Approving  Disapproving  Unsure/NoData  \
0    1941-07-22  07/22/1941         69            24              6   
1    1941-07-29  07/29/1941         65            25              8   
2    1941-08-05  08/05/1941         68            23              7   
3    1941-08-19  08/19/1941         65            26              7   
4    1941-08-26  08/26/1941         67            24              7   
...         ...         ...        ...           ...            ...   
1902 2024-10-01  10/12/2024         39            56              5   
1903 2024-10-14  10/27/2024         41            39              3   
1904 2024-11-06  11/20/2024         37            58              6   
1905 2024-12-02  12/18/2024         39            56              5   
1906 2025-01-02   1/15/2025         40            54              6   

      Candidate    GDP_x    GDP_y  GDP_Change  Income_x  ...  Dividends_x  \
0     roosevelt      NaN      NaN         NaN       NaN  ...          

In [76]:
# take care of NaNs
df.ffill(inplace=True)
df.fillna(method='bfill', inplace=True)  # Backup fill
df.replace([np.inf, -np.inf], np.nan, inplace=True)

In [77]:
# drop rows with missing target values
df.dropna(subset=['Approving'], inplace=True)

In [78]:
print(df.columns)
print(df)

Index(['Start Date', 'End Date', 'Approving', 'Disapproving', 'Unsure/NoData',
       'Candidate', 'GDP_x', 'GDP_y', 'GDP_Change', 'Income_x', 'Income_y',
       'Income_Change', 'Unemployment', 'Open_x', 'High_x', 'Low_x',
       'SP500_Close_x', 'Volume_x', 'Dividends_x', 'Stock Splits_x', 'Open_y',
       'High_y', 'Low_y', 'SP500_Close_y', 'Volume_y', 'Dividends_y',
       'Stock Splits_y', 'SP500_Close_Change'],
      dtype='object')
     Start Date    End Date  Approving  Disapproving  Unsure/NoData  \
0    1941-07-22  07/22/1941         69            24              6   
1    1941-07-29  07/29/1941         65            25              8   
2    1941-08-05  08/05/1941         68            23              7   
3    1941-08-19  08/19/1941         65            26              7   
4    1941-08-26  08/26/1941         67            24              7   
...         ...         ...        ...           ...            ...   
1902 2024-10-01  10/12/2024         39            56        

In [108]:
# use GDP, Unemployment, and SP500_Close as predictors (remove Income and GDP_Unemployment)
X_1 = df[['Income_y']]
X_2 = df[['GDP_y']]
X_3 = df[['SP500_Close_y']]
X_4 = df[['Unemployment']]

X = df[['Income_y', 'GDP_y', 'SP500_Close_y']]  # focus on the most relevant variables-- but maybe  change??

y = df['Approving']-df['Disapproving']

# normalize  predictors
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_1 = scaler.fit_transform(X_1)
X_scaled_2 = scaler.fit_transform(X_2)
X_scaled_3 = scaler.fit_transform(X_3)
X_scaled_4 = scaler.fit_transform(X_4)

# constant term for intercept
X_scaled = sm.add_constant(X_scaled)
X_scaled_1 = sm.add_constant(X_scaled_1)
X_scaled_2 = sm.add_constant(X_scaled_2)
X_scaled_3 = sm.add_constant(X_scaled_3)
X_scaled_4 = sm.add_constant(X_scaled_4)

In [109]:
# fit model
model = sm.OLS(y, X_scaled).fit()
model_1 = sm.OLS(y, X_scaled_1).fit()
model_2 = sm.OLS(y, X_scaled_2).fit()
model_3 = sm.OLS(y, X_scaled_3).fit()
model_4 = sm.OLS(y, X_scaled_4).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.238
Model:                            OLS   Adj. R-squared:                  0.237
Method:                 Least Squares   F-statistic:                     198.6
Date:                Thu, 20 Mar 2025   Prob (F-statistic):          4.82e-112
Time:                        21:44:14   Log-Likelihood:                -8670.0
No. Observations:                1907   AIC:                         1.735e+04
Df Residuals:                    1903   BIC:                         1.737e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         11.8348      0.523     22.627      0.0

In [111]:
# eval model performance
y_pred = model.predict(X_scaled)
mae = mean_absolute_error(y, y_pred)
rmse = np.sqrt(mean_squared_error(y, y_pred))
r2 = r2_score(y, y_pred)

print("\nTotal Model Performance:")
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R-squared: {r2:.2f}")

# eval model 1 performance
y_pred_1 = model_1.predict(X_scaled_1)
mae_1 = mean_absolute_error(y, y_pred_1)
rmse_1 = np.sqrt(mean_squared_error(y, y_pred_1))
r2_1 = r2_score(y, y_pred_1)

print("\nModel 1 Performance - Median Income:")
print(f"MAE: {mae_1:.2f}")
print(f"RMSE: {rmse_1:.2f}")
print(f"R-squared: {r2_1:.2f}")

# eval model 2 performance
y_pred_2 = model_2.predict(X_scaled_2)
mae_2 = mean_absolute_error(y, y_pred_2)
rmse_2 = np.sqrt(mean_squared_error(y, y_pred_2))
r2_2 = r2_score(y, y_pred_2)

print("\nModel 2 Performance - GDP per capita:")
print(f"MAE: {mae_2:.2f}")
print(f"RMSE: {rmse_2:.2f}")
print(f"R-squared: {r2_2:.2f}")

# eval model 3 performance
y_pred_3 = model_3.predict(X_scaled_3)
mae_3 = mean_absolute_error(y, y_pred_3)
rmse_3 = np.sqrt(mean_squared_error(y, y_pred_3))
r2_3 = r2_score(y, y_pred_3)

print("\nModel 3 Performance - S&P500 value:")
print(f"MAE: {mae_3:.2f}")
print(f"RMSE: {rmse_3:.2f}")
print(f"R-squared: {r2_3:.2f}")

# eval model 4 performance
y_pred_4 = model_4.predict(X_scaled_4)
mae_4 = mean_absolute_error(y, y_pred_4)
rmse_4 = np.sqrt(mean_squared_error(y, y_pred_4))
r2_4 = r2_score(y, y_pred_4)

print("\nModel 4 Performance - Unemployment Rate:")
print(f"MAE: {mae_4:.2f}")
print(f"RMSE: {rmse_4:.2f}")
print(f"R-squared: {r2_4:.2f}")


Total Model Performance:
MAE: 17.20
RMSE: 22.82
R-squared: 0.24

Model 1 Performance - Median Income:
MAE: 19.33
RMSE: 24.23
R-squared: 0.14

Model 2 Performance - GDP per capita:
MAE: 17.84
RMSE: 23.36
R-squared: 0.20

Model 3 Performance - S&P500 value:
MAE: 18.54
RMSE: 23.63
R-squared: 0.18

Model 4 Performance - Unemployment Rate:
MAE: 21.40
RMSE: 26.12
R-squared: 0.00
